# Créer et exécuter un pipeline RAG local à partir de zéro

L'objectif de ce notebook est de créer un pipeline RAG (Retrieval Augmented Generation) à partir de zéro et de le faire fonctionner sur un GPU local.

Plus précisément, nous aimerions pouvoir ouvrir un fichier PDF, lui poser des questions (requêtes) et y répondre par un Large Language Model (LLM).

Il existe des frameworks qui reproduisent ce type de flux de travail, notamment [LlamaIndex](https://www.llamaindex.ai/) et [LangChain](https://www.langchain.com/). Cependant, l'objectif de construire à partir de scratch, c'est pouvoir inspecter et personnaliser toutes les pièces.

## Pourquoi RAG ?

L'objectif principal de RAG est d'améliorer le rendement de génération des LLM.

Deux améliorations principales peuvent être considérées comme :
1. **Prévenir les hallucinations** - Les LLM sont incroyables mais ils sont sujets à des hallucinations potentielles, comme générer quelque chose qui *semble* correct mais ne l'est pas. Les pipelines RAG peuvent aider les LLM à générer davantage de résultats factuels en leur fournissant des entrées factuelles (récupérées). Et même si la réponse générée à partir d'un pipeline RAG ne semble pas correcte, grâce à la récupération, vous avez également accès aux sources d'où elle provient.
2. **Travailler avec des données personnalisées** - De nombreux LLM de base sont formés avec des données textuelles à l'échelle Internet. Cela signifie qu’ils ont une grande capacité à modéliser le langage, mais qu’ils manquent souvent de connaissances spécifiques. Les systèmes RAG peuvent fournir aux LLM des données spécifiques à un domaine telles que des informations médicales ou de la documentation d'entreprise et ainsi personnaliser leurs sorties pour répondre à des cas d'utilisation spécifiques.

Les auteurs de l’article original du RAG mentionné ci-dessus ont souligné ces deux points dans leur discussion.

> Ce travail offre plusieurs avantages sociétaux positifs par rapport aux travaux antérieurs : le fait qu'il soit plus
fortement ancré dans des connaissances factuelles réelles (dans ce cas Wikipédia), le rend moins « halluciné »
avec des générations plus factuelles et offre plus de contrôle et d’interprétabilité. RAG pourrait être
employé dans une grande variété de scénarios bénéficiant directement à la société, par exemple en la dotant
avec un index médical et en lui posant des questions ouvertes sur ce sujet, ou en aidant les gens à être plus
efficaces dans leur travail.

RAG peut également être une solution beaucoup plus rapide à mettre en œuvre que d’affiner un LLM sur des données spécifiques.

Pourquoi local ?
Confidentialité, rapidité, coût.

Exécuter localement signifie que vous utilisez votre propre matériel.

Du point de vue de la confidentialité, cela signifie que vous n'avez pas besoin d'envoyer de données potentiellement sensibles à une API.

Du point de vue de la vitesse, cela signifie que vous n'aurez pas nécessairement à attendre une file d'attente d'API ou un temps d'arrêt ; si votre matériel est en cours d'exécution, le pipeline peut s'exécuter.

Et du point de vue des coûts, fonctionner sur votre propre matériel entraîne souvent un coût de départ plus élevé, mais peu ou pas de frais par la suite.

En termes de performances, les API LLM peuvent toujours fonctionner mieux qu'un modèle open source exécuté localement sur des tâches générales, mais il existe de plus en plus d'exemples de modèles plus petits et ciblés surpassant les modèles plus grands.

## Termes clés

| Term | Description |
| ----- | ----- | 
| **Jeton** | Un morceau de texte sous-mot. Par exemple, « Bonjour tout le monde ! » pourrait être divisé en ["hello", ",", "world", "!"]. Un jeton peut être un mot entier,<br> une partie d'un mot ou un groupe de caractères de ponctuation. 1 jeton ~= 4 caractères en anglais, 100 jetons ~= 75 mots.<br> Le texte est divisé en jetons avant d'être transmis à un LLM. |
| **Embedding** | Une représentation numérique apprise d’une donnée. Par exemple, une phrase de texte pourrait être représentée par un vecteur avec<br> 768 valeurs. Des morceaux de texte similaires (dans leur sens) auront idéalement des valeurs similaires. |
| **Embedding model** | Un modèle conçu pour accepter des données d'entrée et produire une représentation numérique. Par exemple, un modèle d'incorporation de texte peut prendre 384 <br>jetons de texte et le transformer en un vecteur de taille 768. Un modèle d'incorporation peut et est souvent différent d'un modèle LLM. |
| **Similarity search/vector search** | Similarity search/vector la recherche vise à trouver deux vecteurs proches l’un de l’autre dans un espace de haute dimension. Par exemple, deux morceaux de texte similaire transmis via un modèle d'intégration devraient avoir un score de similarité élevé, tandis que deux morceaux de texte sur<br> des sujets différents auront un score de similarité plus faible. Les mesures courantes du score de similarité sont la similarité du produit scalaire et du cosinus. |
| **Large Language Model (LLM)** | Un modèle qui a été entraîné pour représenter numériquement les modèles dans le texte. Un LLM génératif continuera une séquence lorsqu'on lui donnera une séquence. <br>Par exemple, étant donné une séquence du texte « bonjour tout le monde ! », un LLM génératif peut produire « nous allons construire un pipeline RAG aujourd'hui ! ».<br> Cette génération sera fortement dépendante de la formation données et invite.|
| **LLM context window** | Le nombre de jetons qu'un LLM peut accepter en entrée. Par exemple, depuis mars 2024, GPT-4 dispose d'une fenêtre contextuelle par défaut de 32 000 jetons<br> (environ 96 pages de texte), mais peut aller jusqu'à 128 000 si nécessaire. Un récent LLM open source de Google, Gemma (mars 2024) a une fenêtre contextuelle<br> de 8 192 jetons (environ 24 pages de texte). Une fenêtre de contexte plus élevée signifie qu'un LLM peut accepter des informations plus pertinentes<br> pour faciliter une requête. Par exemple, dans un pipeline RAG, si un modèle dispose d'une fenêtre contextuelle plus grande, il peut accepter davantage d'éléments de référence<br> du système de récupération pour faciliter sa génération.|
| **Prompt** | Un terme courant pour décrire l'entrée dans un LLM génératif. L'idée de « [prompt Engineering](https://en.wikipedia.org/wiki/Prompt_engineering) » est de structurer une entrée basée sur du texte<br> (ou potentiellement également basée sur des images) dans un LLM génératif dans un manière spécifique afin que la sortie générée soit idéale. Cette technique est<br>possible grâce à la capacité d'un LLM à apprendre en contexte, car il est capable d'utiliser sa représentation du langage pour décomposer l'invite et reconnaître ce que peut être un résultat approprié (remarque : le résultat des LLM est probable, c'est pourquoi des termes tels que « peut produire » sont utilisés).| 




## Ce que nous allons construire

Nous allons créer un pipeline RAG qui nous permet de discuter avec un document PDF, en particulier des pdfs sur les programmes des lycées au Cameroun.

Vous pourriez appeler notre projet EduChat !

Nous écrirons le code dans :
1. Ouvrez un document PDF (vous pouvez utiliser presque n'importe quel PDF ici).
2. Formatez le texte du manuel PDF pour le préparer à un modèle d'Embedding(ce processus est connu sous le nom de fractionnement/blocage de texte).
3. Intégrez tous les morceaux de texte dans le manuel et transformez-les en représentation numérique que nous pourrons stocker pour plus tard.
4. Créez un système de récupération qui utilise la recherche vectorielle pour trouver des morceaux de texte pertinents en fonction d'une requête.
5. Créez une invite qui intègre les morceaux de texte récupérés.
6. Générez une réponse à une requête basée sur des passages du manuel.

Les étapes ci-dessus peuvent être divisées en deux sections principales :
1. Prétraitement/création d'Embedidng des document (étapes 1 à 3).
2. Recherchez et répondez (étapes 4 à 6).

Et c'est la structure que nous suivrons.

C'est similaire au flux de travail décrit sur le blog NVIDIA qui [détaille un pipeline RAG local](https://developer.nvidia.com/blog/rag-101-demystifying-retrieval-augmented-generation-pipelines/).

<img src="https://github.com/mrdbourke/simple-local-rag/blob/main/images/simple-local-rag-workflow-flowchart.png?raw=true" alt="organigramme d'un local Flux de travail RAG" />

## 1. Traitement de documents/textes et création d'Embeddings

Ingrédients:
* Document PDF au choix.
* Modèle d'intégration au choix.

Mesures:
1. Importez un document PDF.
2. Traitez le texte pour l'intégrer (par exemple, divisé en morceaux de phrases).
3. Intégrez des morceaux de texte avec le modèle d'intégration.
4. Enregistrez les intégrations dans un fichier pour une utilisation ultérieure (les intégrations seront stockées dans un fichier pendant de nombreuses années ou jusqu'à ce que vous perdiez votre disque dur).

Importer les document PDF
Cela fonctionnera avec de nombreux autres types de documents.

Il existe plusieurs bibliothèques pour ouvrir des PDF avec Python mais j'ai trouvé que PyMuPDF fonctionne plutôt bien dans de nombreux cas.


In [21]:
import os
from tqdm.auto import tqdm  # Pour afficher une barre de progression
import fitz  # PyMuPDF, pour lire les fichiers PDF

# Fonction pour formater le texte
def formater_texte(texte: str) -> str:
    """Effectue un léger formatage du texte."""
    texte_nettoye = texte.replace("\n", " ").strip()  # Remplace les sauts de ligne par des espaces et supprime les espaces inutiles
    return texte_nettoye

# Fonction pour ouvrir et lire un fichier PDF
def ouvrir_et_lire_pdf(chemin_pdf: str) -> list[dict]:
    doc = fitz.open(chemin_pdf)  # Ouvre le document PDF
    pages_et_textes = []
    for numero_page, page in tqdm(enumerate(doc), desc=f"Traitement de {os.path.basename(chemin_pdf)}"):  # Parcourt les pages
        texte = page.get_text()  # Extrait le texte de la page
        texte = formater_texte(texte)  # Nettoie le texte
        pages_et_textes.append({
            "numero_page": numero_page + 1,  # Les numéros de page commencent à 1
            "nb_caracteres": len(texte),
            "nb_mots": len(texte.split(" ")),
            "nb_phrases": len(texte.split(". ")),
            "nb_tokens_estime": len(texte) / 4,  # Estimation des tokens (1 token ≈ 4 caractères)
            "texte": texte
        })
    return pages_et_textes

# Dossier contenant les fichiers PDF locaux
dossier_pdf = "data"  # Modifiez ceci si votre dossier a un autre nom

# Vérifie si le dossier existe
if not os.path.exists(dossier_pdf):
    raise FileNotFoundError(f"Le dossier '{dossier_pdf}' n'existe pas.")

# Liste tous les fichiers PDF dans le dossier
fichiers_pdf = [os.path.join(dossier_pdf, f) for f in os.listdir(dossier_pdf) if f.endswith(".pdf")]

# Traite tous les fichiers PDF du dossier
contenu_tous_les_pdfs = {}

for fichier_pdf in fichiers_pdf:
    print(f"Lecture du fichier : {fichier_pdf}")
    contenu_pdf = ouvrir_et_lire_pdf(fichier_pdf)  # Lit le contenu du PDF
    contenu_tous_les_pdfs[os.path.basename(fichier_pdf)] = contenu_pdf  # Stocke le contenu avec le nom du fichier comme clé

# Exemple : Affiche les 2 premières pages de chaque PDF traité
for nom_pdf, contenu in contenu_tous_les_pdfs.items():
    print(f"\n--- {nom_pdf} ---")
    for page in contenu[:2]:  # Affiche uniquement les 2 premières pages
        print(f"Page {page['numero_page']} :")
        print(page['texte'])
        print("\n")


Lecture du fichier : data/Physique-Chimie-Technologie-4ème.pdf


Traitement de Physique-Chimie-Technologie-4ème.pdf: 47it [00:01, 44.88it/s]


Lecture du fichier : data/Physics F1-F2.pdf


Traitement de Physics F1-F2.pdf: 38it [00:01, 28.34it/s]


Lecture du fichier : data/Biology Form 3-4-5.pdf


...

Lecture du fichier : data/ECM 4ème,3ème.pdf


Traitement de ECM 4ème,3ème.pdf: 48it [00:00, 71.17it/s]


Lecture du fichier : data/ANGLAIS 4e ESG .pdf


Traitement de ANGLAIS 4e ESG .pdf: 62it [00:00, 93.19it/s]


--- Physique-Chimie-Technologie-4ème.pdf ---
Page 1 :
Page 1 sur 47      DOMAINE D’APPRENTISSAGE : SCIENCES ET TECHNOLOGIE  PROGRAMME D’ÉTUDES : PHYSIQUE-CHIMIE-TECHNOLOGIE  NIVEAU : QUATRIEME  VOLUMES HORAIRES :  VOLUME HORAIRE ANNUEL : 75 HEURES  VOLUME HORAIRE HEBDOMADAIRE : 03 HEURES  COEFFICIENT : 03


Page 2 :
Page 2 sur 47      MODULE 1 : LA MATIÈRE : SES PROPRIÉTES ET SES TRANSFORMATIONS   VOLUME HORAIRE ALLOUÉ AU MODULE : 18 HEURES   PRESENTATION DU MODULE   Ce module comporte trois (03) parties :   ➢ Les propriétés et les caractéristiques de la matière ;   ➢ Les aimants, le champ magnétique terrestre ;   ➢ La notion de réaction chimique et d’élément.   Catégories d’actions :  -Détermination des propriétés caractéristiques de la matière   -Réalisation des transformations chimiques   -Détermination des caractéristiques physiques et chimiques d’un corps     Situation problème :   Les objets qui nous entourent ont des propriétés différentes. Certains peuvent quitter de l’état li

Prenons maintenant un échantillon aléatoire des pages.

In [22]:
# Affiche toutes les pages de tous les PDFs
for nom_pdf, contenu in contenu_tous_les_pdfs.items():
    print(f"\n--- Contenu du fichier : {nom_pdf} ---")
    for page in contenu:
        print(f"\n--- Page {page['numero_page']} ---")
        print(f"Numéro de page : {page['numero_page']}")
        print(f"Contenu : {page['texte']}\n")



--- Contenu du fichier : Physique-Chimie-Technologie-4ème.pdf ---

--- Page 1 ---
Numéro de page : 1
Contenu : Page 1 sur 47      DOMAINE D’APPRENTISSAGE : SCIENCES ET TECHNOLOGIE  PROGRAMME D’ÉTUDES : PHYSIQUE-CHIMIE-TECHNOLOGIE  NIVEAU : QUATRIEME  VOLUMES HORAIRES :  VOLUME HORAIRE ANNUEL : 75 HEURES  VOLUME HORAIRE HEBDOMADAIRE : 03 HEURES  COEFFICIENT : 03


--- Page 2 ---
Numéro de page : 2
Contenu : Page 2 sur 47      MODULE 1 : LA MATIÈRE : SES PROPRIÉTES ET SES TRANSFORMATIONS   VOLUME HORAIRE ALLOUÉ AU MODULE : 18 HEURES   PRESENTATION DU MODULE   Ce module comporte trois (03) parties :   ➢ Les propriétés et les caractéristiques de la matière ;   ➢ Les aimants, le champ magnétique terrestre ;   ➢ La notion de réaction chimique et d’élément.   Catégories d’actions :  -Détermination des propriétés caractéristiques de la matière   -Réalisation des transformations chimiques   -Détermination des caractéristiques physiques et chimiques d’un corps     Situation problème :   Les obj

### Obtenez des statistiques sur le texte

Effectuons une analyse exploratoire approximative des données (EDA) pour avoir une idée de la taille des textes (par exemple, nombre de caractères, nombre de mots, etc.) avec lesquels nous travaillons.

Les différentes tailles de textes seront un bon indicateur de la manière dont nous devrions diviser nos textes.

De nombreux modèles d'intégration ont des limites quant à la taille des textes qu'ils peuvent ingérer, par exemple le modèle [`sentence-transformers`](https://www.sbert.net/docs/pretrained_models.html) [`all-mpnet-base -v2`](https://huggingface.co/sentence-transformers/all-mpnet-base-v2) a une taille d'entrée de 384 jetons.

Cela signifie que le modèle a été entraîné à ingérer et à transformer en textes d'intégration avec 384 jetons (1 jeton ~= 4 caractères ~= 0,75 mots).

Les textes de plus de 384 jetons codés par ce modèle seront automatiquement réduits à 384 jetons, ce qui risque de perdre certaines informations.

Nous en discuterons davantage dans la section intégration.

Pour l'instant, transformons notre liste de dictionnaires en DataFrame et explorons-la.

In [23]:
import random

# Sélectionne un fichier PDF particulier pour l'échantillonnage
nom_pdf_test = list(contenu_tous_les_pdfs.keys())[0]  # Exemple : le premier fichier PDF
pages_and_texts = contenu_tous_les_pdfs[nom_pdf_test]

# Vérifie si `pages_and_texts` contient au moins 3 pages
if len(pages_and_texts) >= 3:
    # Sélectionne 3 pages aléatoires
    pages_aleatoires = random.sample(pages_and_texts, k=3)
    
    # Affiche les pages sélectionnées
    for i, page in enumerate(pages_aleatoires, start=1):
        print(f"\n--- Page Aléatoire {i} ---")
        print(f"Numéro de page : {page['numero_page']}")
        print(f"Contenu : {page['texte']}\n")
else:
    print(f"Le fichier PDF '{nom_pdf_test}' contient moins de 3 pages. Impossible de faire un échantillonnage.")



--- Page Aléatoire 1 ---
Numéro de page : 13
Contenu : Page 13 sur 47     L’intensité d’un courant électrique se mesure à l’aide d’un ampèremètre branché en série.    L’unité de l’intensité est l’ampère (symbole : « A »).  Activité : A l’aide d’un testeur, tester les deux trous (bornes) d’un circuit électrique.  Que constatez-vous ? conclure.   2.2.2. Sens conventionnel du courant électrique ;   A l'extérieur du générateur le courant électrique circule de la borne + à la borne   - du générateur.  2.2.3. Mesure de l’intensité du courant ;   Un ampèremètre possède une borne COM (fil noir) relié à la borne négative du générateur et une autre borne (fil rouge) qu’on peut  brancher sur la borne des 10 A ou des mA.


--- Page Aléatoire 2 ---
Numéro de page : 8
Contenu : Page 8 sur 47    a-A partir de cette phrase, écrire l’équation bilan de la combustion du soufre   b-Donner les réactifs et produit de cette réaction.   c-Est-ce une combustion complète ? justifier.  d- Donner un exemple de c

In [24]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()

,numero_page,nb_caracteres,nb_mots,nb_phrases,nb_tokens_estime,texte
0,1,252,46,1,63.00,Page 1 sur 47 DOMAINE D’APPRENTISSAGE : S...
1,2,1016,186,5,254.00,Page 2 sur 47 MODULE 1 : LA MATIÈRE : SES...
2,3,2255,394,17,563.75,Page 3 sur 47 2- Identifier parmi ces mots ...
3,4,2132,379,17,533.00,Page 4 sur 47 1.2- Les aimants et le champ...
4,5,1330,269,5,332.50,Page 5 sur 47 1.2.4 Utilisation d’une bouss...


In [25]:
# Get stats
df.describe().round(2)

,numero_page,nb_caracteres,nb_mots,nb_phrases,nb_tokens_estime
count,47.00,47.00,47.00,47.00,47.00
mean,24.00,1177.51,222.79,9.57,294.38
std,13.71,513.32,91.16,5.09,128.33
min,1.00,13.00,4.00,1.00,3.25
25%,12.50,785.50,144.50,6.00,196.38
50%,24.00,1246.00,248.00,8.00,311.50
75%,35.50,1499.50,283.00,12.00,374.88
max,47.00,2255.00,394.00,24.00,563.75


D'accord, il semble que notre nombre moyen de jetons par page soit de 294.

### Traitement ultérieur du texte (divisation des pages en phrases)

La manière idéale de traiter le texte avant de l’intégrer reste un domaine de recherche actif.

Une méthode simple que j'ai trouvée utile consiste à diviser le texte en morceaux de phrases.

Comme dans, divisez une page de texte en groupes de 5, 7, 10 phrases ou plus (ces valeurs ne sont pas gravées dans le marbre et peuvent être explorées).

Mais nous voulons suivre le flux de travail de :

`Ingérer du texte -> le diviser en groupes/morceaux -> intégrer les groupes/morceaux -> utiliser les intégrations`

Quelques options pour diviser le texte en phrases :

1. Diviser en phrases avec des règles simples (par exemple diviser sur ". " avec `text = text.split(". ")`, comme nous l'avons fait ci-dessus).
2. Divisez en phrases avec une bibliothèque de traitement du langage naturel (NLP) telle que [spaCy](https://spacy.io/) ou [nltk](https://www.nltk.org/).

Pourquoi diviser en phrases ?

* Plus facile à gérer que des pages de texte plus volumineuses (surtout si les pages sont densément remplies de texte).
* Peut être précis et découvrir quel groupe de phrases a été utilisé pour aider dans un pipeline RAG.

> **Ressource :** Voir [instructions d'installation de spaCy](https://spacy.io/usage). 

Utilisons spaCy pour diviser notre texte en phrases, car c'est probablement un peu plus robuste que d'utiliser simplement `text.split(". ")`.

In [26]:
from spacy.lang.fr import French # see https://spacy.io/usage for install instructions

nlp = French()

# Add a sentencizer pipeline, see https://spacy.io/api/sentencizer/ 
nlp.add_pipe("sentencizer")

# Create a document instance as an example
doc = nlp("Ceci est une phrase. Voici une autre phrase.")
assert len(list(doc.sents)) == 2

# Accéder aux phrases du document
print(list(doc.sents))  # Affiche les phrases détectées


[Ceci est une phrase., Voici une autre phrase.]


Nous n'avons pas nécessairement besoin d'utiliser spaCy, cependant, il s'agit d'une bibliothèque open source conçue pour effectuer des tâches NLP comme celle-ci à grande échelle.

Alors exécutons notre petit pipeline de détermination de peine sur nos pages de texte.

In [27]:
# Divise le texte en phrases pour chaque page dans `pages_and_texts`
# Liste globale pour toutes les pages de tous les PDF
pages_and_texts = []

# Traite tous les fichiers PDF du dossier
for fichier_pdf in fichiers_pdf:
    print(f"Lecture du fichier : {fichier_pdf}")
    contenu_pdf = ouvrir_et_lire_pdf(fichier_pdf)  # Lit le contenu du PDF
    
    # Ajouter les pages du fichier actuel à la liste globale
    pages_and_texts.extend(contenu_pdf)

for item in tqdm(pages_and_texts, desc="Traitement des phrases avec SpaCy"):
    # Extraire les phrases de la page
    item["phrases"] = list(nlp(item["texte"]).sents)

    # Convertir les phrases en chaînes de caractères
    item["phrases"] = [str(phrase) for phrase in item["phrases"]]

    # Compter le nombre de phrases détectées sur la page
    item["nb_phrases_spacy"] = len(item["phrases"])

# Inspection d'une page aléatoire pour vérifier les résultats
page_aleatoire = random.sample(pages_and_texts, k=1)[0]  # Sélectionne une page aléatoire

# Afficher les résultats pour la page sélectionnée
print("\n--- Exemple de page analysée ---")
print(f"Numéro de page : {page_aleatoire['numero_page']}")
print(f"Nombre de phrases détectées (SpaCy) : {page_aleatoire['nb_phrases_spacy']}")
print("Quelques phrases extraites :")
for phrase in page_aleatoire["phrases"][:3]:  # Affiche les 3 premières phrases
    print(f"- {phrase}")

Lecture du fichier : data/Physique-Chimie-Technologie-4ème.pdf


Traitement de Physique-Chimie-Technologie-4ème.pdf: 47it [00:00, 93.40it/s]


Lecture du fichier : data/Physics F1-F2.pdf


Traitement de Physics F1-F2.pdf: 38it [00:00, 41.35it/s]


Lecture du fichier : data/Biology Form 3-4-5.pdf


...

Lecture du fichier : data/ECM 4ème,3ème.pdf


Traitement de ECM 4ème,3ème.pdf: 48it [00:00, 109.08it/s]


Lecture du fichier : data/ANGLAIS 4e ESG .pdf


Traitement de ANGLAIS 4e ESG .pdf: 62it [00:00, 94.55it/s]
Traitement des phrases avec SpaCy: 100%|██████████| 1762/1762 [00:15<00:00, 112.18it/s]


--- Exemple de page analysée ---
Numéro de page : 28
Nombre de phrases détectées (SpaCy) : 3
Quelques phrases extraites :
- 27  B.2.6 TABLE OF MAIN COMPONENTS OF MODULE 2    CONTEXTUALISATION  COMPETENCIES TO BE ATTAINED  RESOURCES  Family of life  situations  Examples of life situations  Categories of  actions  Examples of actions  Basic knowledge  Attitudes  Other  resources  Durati on                      Searching and  communicating   information  through the  Internet                       • Navigating the Internet  • Navigating computer networks  • Simple search, e.g.
- a job, a car, …  • Changes in lifestyle, schooling or  professional  • Traveling arrangements (train, plane)  • New lifestyle  • Comprehension of social issues  • Exploration of a country and its  culture, language, history, and  geography  • Learning with use of a technology  • Upgrading skills  • Interpreting societal issues  • Receiving assistance on homework  • Communication by means of e-mail  • Making commu

Merveilleux!

Transformons maintenant la liste des dictionnaires en DataFrame et obtenons quelques statistiques.

In [28]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,numero_page,nb_caracteres,nb_mots,nb_phrases,nb_tokens_estime,nb_phrases_spacy
count,1762.00,1762.00,1762.00,1762.00,1762.00,1762.00
mean,27.60,1758.83,391.90,9.94,439.71,9.37
std,18.93,1024.12,290.67,10.05,256.03,9.63
min,1.00,0.00,1.00,1.00,0.00,0.00
25%,12.00,999.50,224.25,2.00,249.88,2.00
50%,25.00,1806.00,365.00,7.00,451.50,7.00
75%,40.00,2443.75,517.00,14.00,610.94,13.00
max,91.00,6458.00,2574.00,65.00,1614.50,63.00


Maintenant que notre texte est divisé en phrases, on regroupe ces phrases ?

### Regrouper nos phrases ensemble

Faisons un pas pour diviser notre liste de phrases/texte en morceaux plus petits.

Comme vous l'avez peut-être deviné, ce processus est appelé **chunking**.

Pourquoi faisons-nous cela ?

1. Plus facile de gérer des morceaux de texte de taille similaire.
2. Ne surchargez pas la capacité des modèles d'intégration pour les jetons (par exemple, si un modèle d'intégration a une capacité de 384 jetons, il pourrait y avoir une perte d'informations si vous essayez d'intégrer une séquence de plus de 400 jetons).
3. Notre fenêtre contextuelle LLM (la quantité de jetons qu'un LLM peut accepter) peut être limitée et nécessite de la puissance de calcul, nous voulons donc nous assurer que nous l'utilisons aussi bien que possible.

Il convient de noter qu'il existe de nombreuses façons différentes de créer des morceaux d'informations/de texte.

In [29]:
# Définir la taille des groupes de phrases pour créer des chunks
taille_chunk_phrases = 10 

# Fonction pour diviser récursivement une liste en sous-listes de taille définie
def diviser_liste(liste_entree: list, 
                  taille_sous_liste: int) -> list[list[str]]:
    """
    Divise la liste_entree en sous-listes de taille taille_sous_liste (ou aussi proches que possible).

    Par exemple, une liste de 17 phrases sera divisée en deux sous-listes : [[10], [7]].
    """
    return [liste_entree[i:i + taille_sous_liste] for i in range(0, len(liste_entree), taille_sous_liste)]

# Parcourir les pages et textes pour diviser les phrases en chunks
for item in tqdm(pages_and_texts, desc="Création des chunks de phrases"):
    # Diviser les phrases en groupes
    item["chunks_phrases"] = diviser_liste(liste_entree=item["phrases"],
                                           taille_sous_liste=taille_chunk_phrases)
    
    # Ajouter le nombre total de chunks
    item["nb_chunks"] = len(item["chunks_phrases"])

# Exemple : Affiche le contenu pour une page aléatoire
import random

page_aleatoire = random.sample(pages_and_texts, k=1)[0]  # Sélectionner une page aléatoire

print("\n--- Exemple d'une page avec des chunks ---")
print(f"Numéro de page : {page_aleatoire['numero_page']}")
print(f"Nombre de chunks : {page_aleatoire['nb_chunks']}")
print("Quelques chunks de phrases :")
for i, chunk in enumerate(page_aleatoire["chunks_phrases"][:3], start=1):  # Affiche les 3 premiers chunks
    print(f"\nChunk {i} :")
    for phrase in chunk:
        print(f"- {phrase}")


Création des chunks de phrases: 100%|██████████| 1762/1762 [00:00<00:00, 389294.33it/s]


--- Exemple d'une page avec des chunks ---
Numéro de page : 46
Nombre de chunks : 3
Quelques chunks de phrases :

Chunk 1 :
- Page 46 sur 84  MODULE  6 : ORGANISATION ET GESTION DES DONNÉES  CRÉDIT : 11 heures  Tableau 6 : Classe 5ème  CADRE DE CONTEXTUALISATION  AGIR COMPÉTENT  RESSOURCES  Famille de situations  Exemples de  situations  Catégories  d’actions  Exemples  d’actions  Savoirs  Savoir-faire  Savoir -être  Autres  ressources                          Organisation des  données et  estimation des  quantités dans tous  les domaines de vie.
-        Déplacements  quotidiens.
-  Usage de  médicaments.
-  Pratique d’une  activité de loisir  ou sportive.
-  Achat ou vente  d’un bien de  consommation.
-  Planification de  repas ou  d’activités  agricoles.
-  Participation à  une activité de  formation à  l’école ou en  milieu de travail.
-  Cheptel.
-  Population.
-  Température.

Chunk 2 :
-  Estimation des  quantités.
-                        Traitement  des  informations  comport

In [30]:
# Sélectionner un exemple aléatoire parmi les pages traitées
import random

# Sélectionne une page aléatoire de `pages_and_texts`
page_aleatoire = random.sample(pages_and_texts, k=1)[0]

# Afficher des informations sur la page sélectionnée
print("\n--- Exemple de page avec des chunks ---")
print(f"Numéro de page : {page_aleatoire['numero_page']}")
print(f"Nombre de chunks de phrases : {page_aleatoire['nb_chunks']}")

# Affiche les chunks (chaque chunk contient plusieurs phrases, si la page a plus de 10 phrases)
print("Quelques chunks de phrases :")
for i, chunk in enumerate(page_aleatoire["chunks_phrases"][:3], start=1):  # Affiche les 3 premiers chunks
    print(f"\nChunk {i} :")
    for phrase in chunk:
        print(f"- {phrase}")

# Exemple de sélection d'un chunk aléatoire parmi ceux de cette page
if page_aleatoire["nb_chunks"] > 1:  # Vérifie si la page contient plusieurs chunks
    chunk_aleatoire = random.choice(page_aleatoire["chunks_phrases"])
    print("\n--- Chunk aléatoire sélectionné ---")
    for phrase in chunk_aleatoire:
        print(f"- {phrase}")
else:
    print("\nLa page ne contient qu'un seul chunk.")



--- Exemple de page avec des chunks ---
Numéro de page : 6
Nombre de chunks de phrases : 1
Quelques chunks de phrases :

Chunk 1 :
- Inspection de Pédagogie d’Informatique/Programme selon l’Approche par Compétences/Septembre 2010    Le sommaire  Page iv    IX.
- REFERENTIEL DES COMPETENCES POUR LES CLASSES DE SECONDES  TRONC COMMUN  (ESG – EST-ETN) ............................................................................................................... 41  X. PRESENTATION DES MODULES  POUR LES CLASSES DE SECONDE TRONC COMMUN (ESG  – EST-ETN) ...................................................................................................................... 43  XI.
- REFERENTIEL DES COMPETENCES POUR LES CLASSES DE PREMIERES TRONC  COMMUN (ESG –EST-ETN) ............................................................................................... 48  XII.
- PRESENTATION DES MODULES POUR LES CLASSES DE PREMIERES TRONC  COMMUN (ESG –EST-ETN) ......................................

In [31]:
# Crée un DataFrame à partir des données de `pages_and_texts`
df = pd.DataFrame(pages_and_texts)

# Affiche les statistiques descriptives
df_stats = df.describe().round(2)

# Affiche le DataFrame avec les statistiques
print(df_stats)

       numero_page  nb_caracteres  nb_mots  nb_phrases  nb_tokens_estime  \
count      1762.00        1762.00  1762.00     1762.00           1762.00   
mean         27.60        1758.83   391.90        9.94            439.71   
std          18.93        1024.12   290.67       10.05            256.03   
min           1.00           0.00     1.00        1.00              0.00   
25%          12.00         999.50   224.25        2.00            249.88   
50%          25.00        1806.00   365.00        7.00            451.50   
75%          40.00        2443.75   517.00       14.00            610.94   
max          91.00        6458.00  2574.00       65.00           1614.50   

       nb_phrases_spacy  nb_chunks  
count           1762.00    1762.00  
mean               9.37       1.48  
std                9.63       0.93  
min                0.00       0.00  
25%                2.00       1.00  
50%                7.00       1.00  
75%               13.00       2.00  
max               6

### Diviser chaque morceau en son propre élément

Nous aimerions embedder chaque morceau de phrases dans sa propre représentation numérique.

Donc, pour garder les choses claires, créons une nouvelle liste de dictionnaires contenant chacun un seul morceau de phrases avec des informations relatives telles que le numéro de page ainsi que des statistiques sur chaque morceau.

In [32]:
import re

# Liste pour stocker les chunks traités
pages_et_chunks = []

# Parcourt les pages et les chunks pour créer des données sur chaque chunk
for item in tqdm(pages_and_texts, desc="Traitement des chunks de phrases"):
    for chunk_de_phrases in item["chunks_phrases"]:
        chunk_dict = {}
        chunk_dict["numero_page"] = item["numero_page"]
        
        # Joindre les phrases pour former un "paragraphe" ou chunk
        chunk_texte = "".join(chunk_de_phrases).replace("  ", " ").strip()
        chunk_texte = re.sub(r'\.([A-Z])', r'. \1', chunk_texte)  # Format ".A" -> ". A" pour toute combinaison point/majuscule
        chunk_dict["chunk_texte"] = chunk_texte

        # Calculer les statistiques pour le chunk
        chunk_dict["chunk_nb_caracteres"] = len(chunk_texte)
        chunk_dict["chunk_nb_mots"] = len(chunk_texte.split(" "))
        chunk_dict["chunk_nb_tokens"] = len(chunk_texte) / 4  # Estimation des tokens (1 token ≈ 4 caractères)
        
        # Ajouter le chunk traité à la liste des chunks
        pages_et_chunks.append(chunk_dict)

# Vérifie combien de chunks ont été traités
print(f"Nombre total de chunks traités : {len(pages_et_chunks)}")

Traitement des chunks de phrases: 100%|██████████| 1762/1762 [00:00<00:00, 2875.25it/s]

Nombre total de chunks traités : 2602


In [33]:
# View a random sample
random.sample(pages_et_chunks, k=1)


[{'numero_page': 1,
  'chunk_texte': 'Programme de français deuxième langue : Classes de Form 3, Form 4 et Form 5 Page 1    REPUBLIQUE DU CAMEROUN Paix – Travail – Patrie   REPUBLIC OF CAMEROON Peace – Work - Fatherland   MINISTERE DES ENSEIGNEMENTS SECONDAIRES MINISTRY OF SECONDARY EDUCATION      INSPECTION GENERALE DES ENSEIGNEMENTS INSPECTORATE GENERAL OF EDUCATION                            Observer son environnement pour mieux orienter ses choix de formation et réussir sa vie  INSPECTION DE PEDAGOGIE CHARGÉE DE L’ENSEIGNEMENT ET DE LA PROMOTION DU BILINGUISME INSPECTORATE OF PEDAGOGY IN CHARGE TEACHING AND PROMOTION OF BILINGUALISM PROGRAMMES D’ETUDES DE FORMS III, IV AND V FRANÇAIS 2èmeLANGUE',
  'chunk_nb_caracteres': 669,
  'chunk_nb_mots': 135,
  'chunk_nb_tokens': 167.25}]

**Excellent**!


### Embedder  nos morceaux de texte

In [34]:
# Créer une liste des chunks de texte
chunks_texte = [item["chunk_texte"] for item in pages_et_chunks]


In [35]:
# import csv
# import logging
# from sentence_transformers import SentenceTransformer
# from tqdm import tqdm
# import numpy as np

# # Configurer le logging
# logging.basicConfig(
#     filename="embedding_generation.log",
#     level=logging.INFO,
#     format="%(asctime)s - %(levelname)s - %(message)s",
# )
# logging.info("Début du processus de génération des embeddings.")

# # Charger le modèle SentenceTransformer
# try:
#     model = SentenceTransformer("dunzhang/stella_en_1.5B_v5", trust_remote_code=True)
#     logging.info("Modèle 'dunzhang/stella_en_1.5B_v5' chargé avec succès.")
# except Exception as e:
#     logging.error(f"Erreur lors du chargement du modèle : {str(e)}")
#     raise

# # Liste pour stocker les résultats
# chunks_with_embeddings = []

# # Traiter les chunks pour générer les embeddings
# for chunk in tqdm(pages_et_chunks, desc="Génération des embeddings"):
#     chunk_dict = {}
#     chunk_dict["page_number"] = chunk["numero_page"]
#     chunk_dict["sentence_chunk"] = chunk["chunk_texte"]
#     chunk_dict["chunk_char_count"] = len(chunk["chunk_texte"])
#     chunk_dict["chunk_word_count"] = len(chunk["chunk_texte"].split())
#     chunk_dict["chunk_token_count"] = len(chunk["chunk_texte"]) / 4  # Estimation : 1 token ≈ 4 caractères

#     # Générer les embeddings
#     try:
#         chunk_embedding = model.encode(chunk["chunk_texte"], show_progress_bar=False)
#         chunk_dict["embedding"] = chunk_embedding.tolist()  # Convertir en liste pour le CSV
#         logging.info(f"Embedding généré avec succès pour la page {chunk['numero_page']}.")
#     except Exception as e:
#         logging.error(f"Erreur lors du traitement du chunk de la page {chunk['numero_page']} : {str(e)}")
#         chunk_dict["embedding"] = [0.0] * 1024  # Embedding par défaut en cas d'erreur

#     chunks_with_embeddings.append(chunk_dict)

# # Sauvegarder les embeddings dans un fichier CSV
# output_file = "chunks_embeddings.csv"

# try:
#     with open(output_file, mode="w", encoding="utf-8", newline="") as csvfile:
#         writer = csv.writer(csvfile)

#         # Écrire l'en-tête
#         writer.writerow(["page_number", "sentence_chunk", "chunk_char_count", "chunk_word_count", "chunk_token_count", "embedding"])

#         # Écrire chaque chunk et ses données
#         for chunk_data in chunks_with_embeddings:
#             writer.writerow([
#                 chunk_data["page_number"],
#                 chunk_data["sentence_chunk"],
#                 chunk_data["chunk_char_count"],
#                 chunk_data["chunk_word_count"],
#                 chunk_data["chunk_token_count"],
#                 ";".join(map(str, chunk_data["embedding"]))  # Sérialiser l'embedding en chaîne
#             ])
#     logging.info(f"Les embeddings ont été sauvegardés dans le fichier : {output_file}")
# except Exception as e:
#     logging.error(f"Erreur lors de la sauvegarde dans le fichier CSV : {str(e)}")
#     raise

# print(f"Les embeddings ont été générés et sauvegardés dans le fichier : {output_file}.")
# print("Consultez 'embedding_generation.log' pour les détails des logs.")


In [36]:
# Importer le fichier CSV

text_chunks_and_embedding_df_load = pd.read_csv("chunks_embeddings.csv")
text_chunks_and_embedding_df_load.head(70)




,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,1,REPUBLIQUE DU CAMEROUN Paix – Travail – Patrie...,540,72,135.00,0.5195522308349609;0.8008924722671509;1.845638...
1,3,"PRÉFACE En ce début de millénaire, au moment ...",1707,255,426.75,1.2480679750442505;0.8933440446853638;1.032447...
2,5,LA RÉVISION DES PROGRAMMES D’ÉTUDES DU PREMIER...,1655,240,413.75,1.674798607826233;1.5621140003204346;1.7235532...
3,6,La prise en compte de ces évolutions et de ces...,998,148,249.50,1.0203498601913452;0.6517214179039001;1.281144...
4,8,PROFIL DE SORTIE DU 1er CYCLE Le premier cycle...,1347,214,336.75,0.3749120533466339;0.9573613405227661;0.855241...
...,...,...,...,...,...,...
65,60,b) Méthodologie de l’analyse des documents ...,2026,300,506.50,0.8793312311172485;0.8295912742614746;1.478770...
66,60,quoi ?où ?quand ?pour présenter le document. P...,377,62,94.25,1.3249238729476929;1.0209988355636597;1.648361...
67,61,VII. Enquêtes L’enquête : recherche de témoi...,1519,239,379.75,1.3992677927017212;-0.19110439717769623;1.7639...
68,1,REPUBLIQUE DU CAMEROUN Paix – Travail – Patrie...,528,71,132.00,1.6719698905944824;0.7955226898193359;1.656986...


# RAG - Recherche et Réponse

Nous avons brièvement parlé de RAG au début, mais faisons un rapide récapitulatif.

## Qu'est-ce que RAG ?

RAG signifie **Retrieval Augmented Generation**, ou en français, **Génération Augmentée par Recherche**.  
C'est une méthode qui permet de **rechercher des informations pertinentes** et de **générer une réponse augmentée** basée sur ces informations.

---

### Étapes principales de RAG :

1. **Retrieval (Recherche)**  
   Récupérer les ressources pertinentes pour une question donnée.  
   Exemple :  
   - Si la question est : *"Quels sont les macronutriments ?"*, les résultats devraient contenir des informations sur les protéines, les glucides et les lipides.  

2. **Augmentation**  
   Ajouter les informations récupérées à l'invite utilisée par un modèle de langage (LLM) pour améliorer la précision de la réponse générée.

3. **Generation (Génération)**  
   Le LLM produit une réponse enrichie basée sur les informations pertinentes récupérées. Cela permet d’obtenir une réponse plus correcte et d’avoir des références pour approfondir.

---

### Pourquoi utiliser RAG ?

RAG est utile pour :  
- **Les entreprises avec une grande quantité de documentation** : générer des réponses précises avec des liens vers les ressources.
- **Analyser des emails ou des chaînes de communication** : fournir des réponses aux questions tout en indiquant les sources.

---

## Une analogie : Les LLMs comme calculatrices pour les mots

Avec des **bonnes entrées**, les LLM peuvent transformer des données en **résultats utiles**.

---

### Illustration

Voici une image illustrant le concept de RAG :

![Illustration du concept RAG](https://upload.wikimedia.org/wikipedia/commons/7/79/Data_Visualization_Process.png)

> *Cette image montre comment des données structurées et non structurées peuvent être transformées pour produire des informations exploitables.*

---

## Conclusion

RAG est une méthode puissante pour utiliser des modèles de langage de manière plus précise et factuelle. Cela commence par une **bonne recherche** suivie d’une **génération augmentée**.


# Similarity Search

**Similarity search**, aussi appelée recherche sémantique ou recherche vectorielle, repose sur l'idée de rechercher selon une *"vibe"*.  

Si cela semble abstrait, il s'agit en réalité d'une recherche basée sur le **sens** ou la **signification**.

---

## Différence entre recherche par mot-clé et recherche sémantique

Avec une recherche par **mot-clé**, vous essayez de faire correspondre le mot "apple" avec "apple".  

Cependant, avec une recherche **sémantique/similaire**, vous pourriez rechercher *"macronutrients functions"* et obtenir des résultats qui ne contiennent pas nécessairement les mots *"macronutrients functions"*, mais qui correspondent à leur signification.  

---

### Exemple

Avec une recherche sémantique sur nos données de manuels scolaires et une requête *"macronutrients functions"*, le premier résultat pourrait être :  

> **Résultat :**  
> *Il existe trois classes de macronutriments : les glucides, les lipides et les protéines. Ceux-ci peuvent être métaboliquement transformés en énergie cellulaire. L'énergie des macronutriments provient de leurs liaisons chimiques. Cette énergie chimique est convertie en énergie cellulaire, qui est ensuite utilisée pour effectuer du travail, permettant à nos corps de remplir leurs fonctions de base.*  

Comment c'est cool, n'est-ce pas ?

---

## Lien avec Google et vos propres données

Si vous avez déjà utilisé Google, vous connaissez probablement ce genre de flux de travail.  

Mais ici, l’objectif est d’effectuer ce type de recherche **au sein de vos propres données**.

---

### Préparer vos embeddings

Pour commencer, importons les embeddings que nous avons créés précédemment (tk - lien vers le fichier d'embeddings) et préparons-les à être utilisés en les transformant en tenseur.

---

### Illustration

![Recherche sémantique illustrée](https://upload.wikimedia.org/wikipedia/commons/5/59/Word_embedding_visualization.png)

> *Cette image illustre comment les vecteurs permettent de regrouper des termes similaires dans un espace vectoriel.*

---

## En résumé

La recherche sémantique offre une approche puissante pour trouver des informations pertinentes en fonction de leur sens, plutôt qu'en se basant uniquement sur des correspondances exactes de mots.


In [2]:
import random
import torch
import numpy as np
import pandas as pd

# Vérification du périphérique à utiliser : GPU (cuda) ou CPU
device = "cuda" if torch.cuda.is_available() else "cpu"

# Importer le DataFrame contenant les textes et les embeddings
text_chunks_and_embedding_df = pd.read_csv("chunks_embeddings.csv")

# Convertir la colonne d'embeddings en tableau NumPy
# (La colonne a été convertie en chaîne de caractères lorsqu'elle a été sauvegardée au format CSV)
text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(
    lambda x: np.fromstring(x.strip("[]"), sep=" ")  # Suppression des crochets et conversion en tableau
)

# Vérification de la taille d'un embedding pour s'assurer qu'ils ont bien la forme attendue
print(f"Exemple d'embedding : {text_chunks_and_embedding_df['embedding'].iloc[0]}")
print(f"Taille de l'embedding : {text_chunks_and_embedding_df['embedding'].iloc[0].shape}")

# Utilisation de vstack pour empiler les embeddings et créer une matrice 2D
embeddings = torch.tensor(
    np.vstack(text_chunks_and_embedding_df["embedding"].values),  # Assurez-vous de bien empiler les vecteurs
    dtype=torch.float32
).to(device)

# Afficher la forme du tenseur d'embeddings
print(f"Forme du tenseur d'embeddings : {embeddings.shape}")


Exemple d'embedding : [0.51955223]
Taille de l'embedding : (1,)
Forme du tenseur d'embeddings : torch.Size([2602, 1])


/var/folders/lf/54wtk09x751cz6x7x5pv853w0000gp/T/ipykernel_47232/2546745351.py:15: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  lambda x: np.fromstring(x.strip("[]"), sep=" ")  # Suppression des crochets et conversion en tableau


In [4]:
from sentence_transformers import SentenceTransformer, util
import torch
import pandas as pd
import numpy as np

# Charger le modèle d'embedding
embedding_model = SentenceTransformer("dunzhang/stella_en_1.5B_v5", trust_remote_code=True)

# Charger les embeddings depuis le fichier CSV
text_chunks_and_embedding_df = pd.read_csv("chunks_embeddings.csv")

# Convertir la colonne d'embeddings en tableau NumPy
text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(
    lambda x: np.fromstring(x.strip("[]"), sep=" ")  # Conversion en vecteurs
)

# Convertir en tenseur PyTorch
embeddings = torch.tensor(np.array(text_chunks_and_embedding_df["embedding"].tolist()), dtype=torch.float32).to("cuda" if torch.cuda.is_available() else "cpu")

# 1. Définir la requête
query = "macronutrients functions"
query_embedding = embedding_model.encode(query, convert_to_tensor=True).to("cuda" if torch.cuda.is_available() else "cpu")

# 2. Calculer les scores de similarité avec le produit scalaire
dot_scores = util.dot_score(a=query_embedding, b=embeddings)[0]

# 3. Récupérer les indices des documents les plus pertinents (par exemple, top 3)
top_k = 3
top_results_dot_product = torch.topk(dot_scores, k=top_k)

# Récupérer les passages correspondants aux indices des top-k résultats
top_results = [text_chunks_and_embedding_df.iloc[i] for i in top_results_dot_product.indices.tolist()]

# Afficher les documents récupérés
for i, result in enumerate(top_results):
    print(f"Document {i+1} : {result['text']}")

# Maintenant, tu peux utiliser ces documents pour les envoyer à un modèle génératif comme GPT ou autre pour générer une réponse.


/var/folders/lf/54wtk09x751cz6x7x5pv853w0000gp/T/ipykernel_47232/3929904503.py:14: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  lambda x: np.fromstring(x.strip("[]"), sep=" ")  # Conversion en vecteurs


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x1024 and 1x2602)